In [1]:
import pathlib
import os
import numpy as np
import pandas as pd
from ase.visualize import view
from ase.build.tools import sort as ase_sort
from ase.atoms import Atoms

from aiida import orm, load_profile
from aiida.orm import load_node
from aiida.engine import submit

from qe_tools import CONSTANTS

from aiida_quantumespresso_hp.workflows.hubbard import SelfConsistentHubbardWorkChain
from aiida_quantumespresso.data.hubbard_structure import HubbardStructureData

from project_settings import *

load_profile()

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [60]:
fully_lithiated_df = pd.read_pickle(os.path.join(project_dir, 'data', 'fully_lithiated_df.pkl'))
mn_olivine_ase = fully_lithiated_df['ase_in'].values[0]

sorting_dict = {
    'Mn': 0,
    'O': 1,
    'P': 2,
    'Li': 3,
}

# ! Does not take into account initial magmoms for now
mn_olivine_ase_ordered = Atoms(sorted(mn_olivine_ase, key=lambda x: sorting_dict[x.symbol]))
mn_olivine_ase_ordered.set_cell(mn_olivine_ase.get_cell())

mn_olivine_aiida = orm.StructureData(ase=mn_olivine_ase_ordered)

## HubbardStructureData initialization
Let's initialize the HubbardStructureData with the olivine structure!

In [61]:
hubbard_data_inv = HubbardStructureData(structure=mn_olivine_aiida)

In [62]:
# hubbard_data.reorder_atoms()

## Initializing the on-site Hubbard
Let's initialize the on-site Hubbard parameter for the titanium atom.

In [63]:
# Taken from parameters.in of:
# /home/jgeiger/projects/bat_uv_ml/data/olivine_iurii/LixMnPO4/Li1.00/DFT_plus_UV/9_PDOS/LMPO.scf.1.in
hubbard_data_inv.initialize_onsites_hubbard('Mn', '3d', 4.5618)

Here how it is stored in the class.

In [64]:
hubbard_data_inv.hubbard_parameters

[[0, '3d', 0, '3d', 4.5618, [0, 0, 0], 'dudarev'],
 [1, '3d', 1, '3d', 4.5618, [0, 0, 0], 'dudarev'],
 [2, '3d', 2, '3d', 4.5618, [0, 0, 0], 'dudarev'],
 [3, '3d', 3, '3d', 4.5618, [0, 0, 0], 'dudarev']]

## Initializing the inter-site Hubbard
Let's initialize the inter-site Hubbard parameter for the titanium and oxygen atoms.

In [65]:
hubbard_data_inv.initialize_intersites_hubbard('Mn', '3d', 'O', '2p', 0.0001, number_of_neighbours=7) 
# ! Ti has 6 oxygen neighbors in BaTiO3. Similarly, Mn has 6 oxygen neighbors in LMPO (olivine). So, we need to use N+1 in the call to initialize_intersites_hubbard, as the value is used for list slicing, like [:number_of_neighbours], which is exclusive of the last index.

The parameters are saved in the property `hubbard_parameters` as a list.

In [66]:
hubbard_data_inv.hubbard_parameters

[[0, '3d', 0, '3d', 4.5618, [0, 0, 0], 'dudarev'],
 [1, '3d', 1, '3d', 4.5618, [0, 0, 0], 'dudarev'],
 [2, '3d', 2, '3d', 4.5618, [0, 0, 0], 'dudarev'],
 [3, '3d', 3, '3d', 4.5618, [0, 0, 0], 'dudarev'],
 [0, '3d', 12, '2p', 0.0001, [0, -1, 0], 'dudarev'],
 [0, '3d', 13, '2p', 0.0001, [0, 0, 0], 'dudarev'],
 [0, '3d', 8, '2p', 0.0001, [0, 0, 1], 'dudarev'],
 [0, '3d', 4, '2p', 0.0001, [0, 0, 0], 'dudarev'],
 [0, '3d', 17, '2p', 0.0001, [0, 0, 1], 'dudarev'],
 [0, '3d', 16, '2p', 0.0001, [0, 0, 1], 'dudarev'],
 [0, '3d', 7, '2p', 0.0001, [0, 0, 0], 'dudarev'],
 [1, '3d', 15, '2p', 0.0001, [0, 0, 0], 'dudarev'],
 [1, '3d', 14, '2p', 0.0001, [0, -1, 0], 'dudarev'],
 [1, '3d', 11, '2p', 0.0001, [0, 0, 0], 'dudarev'],
 [1, '3d', 7, '2p', 0.0001, [0, 0, 0], 'dudarev'],
 [1, '3d', 19, '2p', 0.0001, [0, 0, 0], 'dudarev'],
 [1, '3d', 18, '2p', 0.0001, [0, 0, 0], 'dudarev'],
 [1, '3d', 4, '2p', 0.0001, [1, 0, 0], 'dudarev'],
 [2, '3d', 16, '2p', 0.0001, [0, 1, 0], 'dudarev'],
 [2, '3d', 17, '2p'

In [67]:
print(hubbard_data_inv.get_quantum_espresso_hubbard_card())
print(hubbard_data_inv.get_quantum_espresso_hubbard_parameters())

HUBBARD	ortho-atomic
 U	Mn-3d	4.5618
 V	Mn-3d	O-2p	1	321	0.0001
 V	Mn-3d	O-2p	1	14	0.0001
 V	Mn-3d	O-2p	1	401	0.0001
 V	Mn-3d	O-2p	1	5	0.0001
 V	Mn-3d	O-2p	1	410	0.0001
 V	Mn-3d	O-2p	1	409	0.0001
 V	Mn-3d	O-2p	1	8	0.0001
 V	Mn-3d	O-2p	2	16	0.0001
 V	Mn-3d	O-2p	2	323	0.0001
 V	Mn-3d	O-2p	2	12	0.0001
 V	Mn-3d	O-2p	2	8	0.0001
 V	Mn-3d	O-2p	2	20	0.0001
 V	Mn-3d	O-2p	2	19	0.0001
 V	Mn-3d	O-2p	2	621	0.0001
 V	Mn-3d	O-2p	3	465	0.0001
 V	Mn-3d	O-2p	3	18	0.0001
 V	Mn-3d	O-2p	3	11	0.0001
 V	Mn-3d	O-2p	3	7	0.0001
 V	Mn-3d	O-2p	3	13	0.0001
 V	Mn-3d	O-2p	3	14	0.0001
 V	Mn-3d	O-2p	3	146	0.0001
 V	Mn-3d	O-2p	4	20	0.0001
 V	Mn-3d	O-2p	4	467	0.0001
 V	Mn-3d	O-2p	4	374	0.0001
 V	Mn-3d	O-2p	4	6	0.0001
 V	Mn-3d	O-2p	4	380	0.0001
 V	Mn-3d	O-2p	4	379	0.0001
 V	Mn-3d	O-2p	4	7	0.0001

 # Atom 1  Atom 2  Hubbard V (eV)
	1	1	4.5618
	2	2	4.5618
	3	3	4.5618
	4	4	4.5618
	1	321	0.0001
	1	14	0.0001
	1	401	0.0001
	1	5	0.0001
	1	410	0.0001
	1	409	0.0001
	1	8	0.0001
	2	16	0.0001
	2	323	0.0001
	2	12	0.0001
	2	8	0.0001
	

In [72]:
print(type(mn_olivine_aiida))
print(mn_olivine_aiida.get_ase().get_chemical_symbols())
from aiida.orm.nodes.data.structure import StructureData

<class 'aiida.orm.nodes.data.structure.StructureData'>
['Mn', 'Mn', 'Mn', 'Mn', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'P', 'P', 'P', 'P', 'Li', 'Li', 'Li', 'Li']


In [73]:
hubbard_data_inv.store()
hubbard_data_inv.pk

<HubbardStructureData: uuid: d3a59ee6-27a1-4447-9aa4-449aa6ecf7f8 (pk: 3015)>

3015

In [75]:
inverted_hubbard_data_pk = 3015

## Running the `SelfConsistentHubbardWorkChain`

In [76]:
hubbard_data_inv = orm.load_node(inverted_hubbard_data_pk)  # I load the node from the database instead of always regenerating it
hubbard_data_inv.hubbard_parameters
# print(hubbard_data.get_quantum_espresso_hubbard_card())
# type(hubbard_data)

# ? On-site gets only stored once, if I am not mistaken.
# ? Index is changed from 0-based (Python) to 1-based (Quantum ESPRESSO)

[[0, '3d', 0, '3d', 4.5618, [0, 0, 0], 'dudarev'],
 [1, '3d', 1, '3d', 4.5618, [0, 0, 0], 'dudarev'],
 [2, '3d', 2, '3d', 4.5618, [0, 0, 0], 'dudarev'],
 [3, '3d', 3, '3d', 4.5618, [0, 0, 0], 'dudarev'],
 [0, '3d', 12, '2p', 0.0001, [0, -1, 0], 'dudarev'],
 [0, '3d', 13, '2p', 0.0001, [0, 0, 0], 'dudarev'],
 [0, '3d', 8, '2p', 0.0001, [0, 0, 1], 'dudarev'],
 [0, '3d', 4, '2p', 0.0001, [0, 0, 0], 'dudarev'],
 [0, '3d', 17, '2p', 0.0001, [0, 0, 1], 'dudarev'],
 [0, '3d', 16, '2p', 0.0001, [0, 0, 1], 'dudarev'],
 [0, '3d', 7, '2p', 0.0001, [0, 0, 0], 'dudarev'],
 [1, '3d', 15, '2p', 0.0001, [0, 0, 0], 'dudarev'],
 [1, '3d', 14, '2p', 0.0001, [0, -1, 0], 'dudarev'],
 [1, '3d', 11, '2p', 0.0001, [0, 0, 0], 'dudarev'],
 [1, '3d', 7, '2p', 0.0001, [0, 0, 0], 'dudarev'],
 [1, '3d', 19, '2p', 0.0001, [0, 0, 0], 'dudarev'],
 [1, '3d', 18, '2p', 0.0001, [0, 0, 0], 'dudarev'],
 [1, '3d', 4, '2p', 0.0001, [1, 0, 0], 'dudarev'],
 [2, '3d', 16, '2p', 0.0001, [0, 1, 0], 'dudarev'],
 [2, '3d', 17, '2p'

In [77]:
pw_code = orm.load_code('qe-dev-pw@lumi-small')
hp_code = orm.load_code('qe-dev-hp@lumi-small')

builder = SelfConsistentHubbardWorkChain.get_builder_from_protocol(
    pw_code=pw_code,
    hp_code=hp_code,
    hubbard_structure=hubbard_data_inv,
    protocol='fast',
    overrides=pathlib.Path('hubbard_overrides.yaml')
)

builder.skip_first_relax = True

In [78]:
olivine_workchain_submit_inv = submit(builder)
print(olivine_workchain_submit_inv)

uuid: 41c8ad0c-02b3-4583-be16-9d8c6a9ae8a9 (pk: 3040) (aiida.workflows:quantumespresso.hp.hubbard)


In [19]:
olivine_workchain_submit_inv_pk_first = 2914
olivine_workchain_submit_inv_pk_second = 2978
olivine_workchain_submit_inv_pk_second = 3040

In [9]:
!verdi process status 3040
!verdi process report 3040
!verdi process status 3071

SelfConsistentHubbardWorkChain<3040> Waiting [1:while_(should_run_iteration)(5:run_hp)]
    ├── PwBaseWorkChain<3042> Finished [0] [4:results]
    │   ├── create_kpoints_from_distance<3043> Finished [0]
    │   ├── PwCalculation<3047> Finished [410]
    │   ├── PwCalculation<3055> Finished [410]
    │   └── PwCalculation<3063> Finished [0]
    └── HpWorkChain<3069> Waiting [None]
        └── HpBaseWorkChain<3071> Waiting [2:while_(should_run_process)]
            └── HpCalculation<3073> Waiting
2023-03-01 10:20:34 [348 | REPORT]: [3040|SelfConsistentHubbardWorkChain|setup]: system is treated to be non-magnetic because `nspin == 1` in `scf.pw.parameters` input.
2023-03-01 10:20:35 [349 | REPORT]: [3040|SelfConsistentHubbardWorkChain|should_run_relax]: skip_first_relax has been set True; skipping first relaxion...
2023-03-01 10:20:35 [350 | REPORT]: [3040|SelfConsistentHubbardWorkChain|run_scf_smearing]: launching PwBaseWorkChain<3042> with smeared occupations
2023-03-01 10:20:37 [351 | 

In [10]:
!verdi process status 3071
!verdi process report 3071
!verdi process show 3071

HpBaseWorkChain<3071> Waiting [2:while_(should_run_process)]
    └── HpCalculation<3073> Waiting
2023-03-01 10:50:16 [371 | REPORT]: [3071|HpBaseWorkChain|run_process]: launching HpCalculation<3073> iteration #1
Property     Value
-----------  ------------------------------------
type         HpBaseWorkChain
state        Waiting
pk           3071
uuid         2569fcbc-71cb-4ee7-a394-ba65b363e011
label
description
ctime        2023-03-01 10:50:14.283885+01:00
mtime        2023-03-01 10:50:16.442401+01:00

Inputs                 PK    Type
---------------------  ----  --------------------
hp
    code               2183  InstalledCode
    hubbard_structure  3015  HubbardStructureData
    parameters         3030  Dict
    parent_scf         3064  RemoteData
    qpoints            3029  KpointsData
    settings           3031  Dict
clean_workdir          3032  Bool
max_iterations         3033  Int
only_initialization    3070  Bool

Caller             PK  Type
---------------  ----  --------

In [6]:
%%bash
# verdi process show 2914
# printf '=%.0s' {1..100}
# echo
# verdi process status 2914
# printf '=%.0s' {1..100}
# echo
verdi process status 2914
printf '=%.0s' {1..100}
echo

SelfConsistentHubbardWorkChain<2914> Finished [404] [1:while_(should_run_iteration)(6:inspect_hp)]
    ├── PwBaseWorkChain<2916> Finished [0] [4:results]
    │   ├── create_kpoints_from_distance<2917> Finished [0]
    │   ├── PwCalculation<2921> Finished [410]
    │   ├── PwCalculation<2929> Finished [410]
    │   └── PwCalculation<2937> Finished [0]
    └── HpWorkChain<2943> Finished [300] [1:inspect_workchain]
        └── HpBaseWorkChain<2945> Finished [300] [2:while_(should_run_process)(1:inspect_process)]
            └── HpCalculation<2947> Finished [312]


In [104]:
!verdi process show 2943

Property     Value
-----------  -----------------------------------------
type         HpWorkChain
state        Finished [300] A child work chain failed.
pk           2943
uuid         c6f4aa72-a87c-4155-8166-aaccbdf048a7
label
description
ctime        2023-02-28 15:15:55.075488+01:00
mtime        2023-02-28 15:19:11.256668+01:00

Inputs                 PK    Type
---------------------  ----  --------------------
hp
    code               2183  InstalledCode
    parameters         2904  Dict
    qpoints            2903  KpointsData
    settings           2905  Dict
    parent_scf         2938  RemoteData
    hubbard_structure  2889  HubbardStructureData
clean_workdir          2906  Bool
max_iterations         2907  Int
parallelize_atoms      2908  Bool

Caller             PK  Type
---------------  ----  ------------------------------
iteration_01_hp  2914  SelfConsistentHubbardWorkChain

Called             PK  Type
---------------  ----  ---------------
iteration_01_hp  2945  HpBaseWor

In [49]:
%%bash
# Previous erroneous workchain, where Hubbard atoms were not in the beginning,
# which causes a problem in the Hubbard code.
verdi process show 2860
printf '=%.0s' {1..100}
echo
verdi process status 2860
printf '=%.0s' {1..100}
echo
verdi process report 2860
printf '=%.0s' {1..100}
echo

Property     Value
-----------  ----------------------------------------------------------------
type         SelfConsistentHubbardWorkChain
state        Finished [404] The HpWorkChain sub process failed in iteration 1
pk           2860
uuid         89f3b9b9-578e-4b6b-b568-fd60e4f66a82
label
description
ctime        2023-02-27 12:52:42.648723+01:00
mtime        2023-02-27 19:15:58.656121+01:00

Inputs                               PK    Type
-----------------------------------  ----  --------------------
hubbard
    hp
        code                         2183  InstalledCode
        qpoints                      2849  KpointsData
        parameters                   2850  Dict
        settings                     2851  Dict
    clean_workdir                    2852  Bool
    max_iterations                   2853  Int
    parallelize_atoms                2854  Bool
relax
    base
        pw
            pseudos
                Li                   34    UpfData
                O          

In [112]:
import importlib
from aiida_quantumespresso_hp.calculations.functions import structure_relabel_kinds
importlib.reload(structure_relabel_kinds)


hubbard_data = orm.load_node(2889)  # I load the node from the database instead of always regenerating it
failed_hp_workchain = load_node(2945)

qe_hubbard_card_before = hubbard_data.get_quantum_espresso_hubbard_card()
# print('Hubbard card before structure_relabel_kinds\n\n', qe_hubbard_card_before)

relabel_output = structure_relabel_kinds.structure_relabel_kinds(
    hubbard_structure=failed_hp_workchain.outputs.hubbard_structure,
    hubbard=failed_hp_workchain.outputs.hubbard
)

qe_hubbard_card_after = relabel_output['hubbard_structure'].get_quantum_espresso_hubbard_card()

print('Hubbard card after structure_relabel_kinds\n\n', qe_hubbard_card_after)

<module 'aiida_quantumespresso_hp.calculations.functions.structure_relabel_kinds' from '/home/jgeiger/devel/aiida-quantumespresso-hp/src/aiida_quantumespresso_hp/calculations/functions/structure_relabel_kinds.py'>

NotExistentAttributeError: Node<2945> does not have an output with link label 'hubbard_structure'